In [1]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd

from sklearn.preprocessing import QuantileTransformer
from joblib import load

# Load pipeline

In [2]:
pipe = load("mypipe.joblib")

# Get data

In [3]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 

conn = create_engine(string).connect()

In [4]:
df = pd.read_sql(
    """
    select ticker, bm, mom12m, roeq, siccd
    from ghz
    order by ticker
    """,
    conn
)
df = df.dropna()
df = df.set_index("ticker")
features = ["bm", "mom12m", "roeq"]

# If you cannot access the Rice database

Execute the following to test the subsequent code if you cannot access the Rice database.  Eventually, another solution must be found.

In [ ]:
df = pd.read_csv("ghz.csv")
features = ["bm", "mom12m", "roeq"]

# Quantile transform

In [22]:
qt = QuantileTransformer(output_distribution="normal")
df[features] = qt.fit_transform(df[features])

# Define industry variable

In [23]:
inds = pd.read_csv("siccodes12.csv", index_col="industry")
ind_names = inds.index.unique().to_list()

def industry(sic):
  try:
    return inds[(inds.start<=sic)&(sic<=inds.end)].index[0]
  except:
    return "Other"
    
df["industry"] = df.siccd.map(industry)
features.append("industry")

# Make predictions

In [24]:
df["predict"] = pipe.predict(df[features])
df.predict.head()

ticker
A       0.016892
AA     -0.065113
AADI   -0.082670
AAN    -0.073371
AAON    0.056878
Name: predict, dtype: float64